In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
  "What is the distance between {country_a} and {country_b}"
)
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# chat.predict(prompt)

messages = ChatPromptTemplate.from_messages([
  ("system", "You are geography export. And you only reply in {language}."),
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = messages.format_messages(
  language="Greek",
  name="Plato",
  country_a="South Korea",
  country_b="Greece"
)

chat.predict_messages(prompt)

### BaseOutputParser


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    item = text.strip().split(",")
    return list(map(str.strip, item))
  
p = CommaOutputParser()
p.parse("Hello, how, are, you")

template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else"),
  ("human", "{question}")
])

prompt = template.format_messages(
  max_items=10,
  question="What are the colors"
)

result = chat.predict_messages(prompt)
p.parse(result.content)

### LangChain Expression Language(LCEL) : Chain


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    item = text.strip().split(",")
    return list(map(str.strip, item))

template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else"),
  ("human", "{question}")
])

# 입력 => chat model => outputparser 순으로 실행
chain = template | chat | CommaOutputParser()

chain.invoke({
  "max_items":5,
  "question": "What are the pokemons? please reply in korean"
})


### LCEL : Chain과 Chain 연결


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specilaized on making traditional recipes vegerarian. You find altenative ingredients and explain their preparation. You don't radically modify the recipe. If there in no alternative for a food just say you don't know how to replace it"),
  ("human", "{recipe}")
])

# Runnable Map
veg_chain = veg_chef_prompt | chat

final_chains = {"recipe":chef_chain} | veg_chain
final_chains.invoke({
  "cuisine" : "French"
})